LinkedIn Data Analyst Jobs | ETL

Extract info from LinkedIn job postings using Selenium and Beautiful Soup
Search Criteria:
    -Data Analyst
    -Chicago, IL
    -Posted in past 24 hours
    -Within 25 miles
    -Entry level

Import

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import config
from google.cloud import bigquery
from google.oauth2 import service_account

In [4]:
#start timer
start_time = time.perf_counter()

Extract

In [42]:
#use Selenium to scroll to bottom of search results page and begin extracting data
#url to extract
url  = 'https://www.linkedin.com/jobs/search?keywords=Data%20Analyst&location=Chicago%2C%20Illinois%2C%20United%20States&locationId=&geoId=103112676&f_TPR=r86400&distance=25&f_E=2&position=1&pageNum=0'

In [43]:
def extract_all_jobs(url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    #scroll to bottom
    SCROLL_PAUSE_TIME = 0.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    class_name = "job-search-card"
    jobs = driver.find_elements(By.CLASS_NAME,class_name)
    for job in jobs:
        job_link = job.find_element(By.CLASS_NAME,"base-card__full-link").get_attribute('href')
        title = job.find_element(By.CLASS_NAME,"base-search-card__title").text
        company = job.find_element(By.CLASS_NAME,"hidden-nested-link").text
        company_link = job.find_element(By.CLASS_NAME,"hidden-nested-link").get_attribute('href')
        location = job.find_element(By.CLASS_NAME,"job-search-card__location").text
        try:
            benefit = job.find_element(By.CLASS_NAME,"result-benefits__text").text
        except:
            benefit = ''
        data ={
            'date' : date.today(),
            'job_link': job_link,
            'title': title,
            'company': company,
            'company_link': company_link,
            'location': location,
            'benefit': benefit
        }
        job_list.append(data)
        
    driver.close() #close the browser
    return

In [44]:
job_list = []
extract_all_jobs(url)
df = pd.DataFrame(job_list) #create dataframe with job list

In [48]:
#use Beautiful Soup to scrape each job link of more info
def extract_job_details(job_links):
    for link in job_links:
        #headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
        url  = link
        #r = requests.get(url,headers)
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver.get(url)
        #wait for page load
        try:
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'description__job-criteria-list'))
            )
        except:
            continue
        #driver.implicitly_wait(10)
        
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser') #read with beautiful soup
        #use try loops for each column. when page doesn't load, this will prevent errors and keep row count the same
        try:
            description = soup.find('div',class_ ='show-more-less-html__markup').text.strip()
        except:
            description = ''
        try:
            applicant_caption = soup.find('figcaption',class_ ='num-applicants__caption').text.strip()
        except:
            applicant_caption = ''
        try:
            posted_time_ago = soup.find('span',class_ ='posted-time-ago__text posted-time-ago__text--new topcard__flavor--metadata').text.strip()
        except:
            posted_time_ago = ''
        criteria = soup.find_all('li',class_='description__job-criteria-item')
        try:
            seniority_level = criteria[0].find('span',class_ = 'description__job-criteria-text description__job-criteria-text--criteria').text.strip()
        except:
            seniority_level = ''
        try:
            employment_type = criteria[1].find('span',class_ = 'description__job-criteria-text description__job-criteria-text--criteria').text.strip()
        except:
            employment_type = ''
        try:
            job_function = criteria[2].find('span',class_ = 'description__job-criteria-text description__job-criteria-text--criteria').text.strip()
        except:
            job_function = ''
        try:
            industries = criteria[3].find('span',class_ = 'description__job-criteria-text description__job-criteria-text--criteria').text.strip()
        except:
            industries = ''
            
            
        #extract additional data from large block of text at bottom of page
        
        try:
            data = json.loads(soup.find('script',{'type':'application/ld+json'}).text)
        except:
            data = ''
        #either block of text does not exist for all links, or some links do not work
        #will use try loops for all columns so number of rows match extract_all_jobs
        try:
            date_posted = data['datePosted'] #begin extracting data using json format
        except:
            date_posted = ''
        try:
            description = data['description']
        except:
            description = ''
        try:
            employment_type = data['employmentType']
        except:
            employment_type = ''
        try:
            industry = data['industry']
        except:
            industry = ''
        try:
            latitude = data['jobLocation']['latitude']
        except:
            latitude = ''
        try:
            longitude = data['jobLocation']['longitude']
        except:
            longitude = ''
        try:
            education_type = data['educationRequirements']['@type']
        except:
            education_type = ''
        try:
            education_category = data['educationRequirements']['credentialCategory']
        except:
            education_category = ''
        try:
            experience_type = data['experienceRequirements']['@type']
        except:
            experience_type = ''
        try:
            experience_time = data['experienceRequirements']['monthsOfExperience']
        except:
            experience_time = ''
        try:
            job_location_type = data['jobLocationType']
        except:
            job_location_type = ''
        try:
            location_requirement_type = data['applicantLocationRequirements']['@type']
        except:
            location_requirement_type = ''
        try:
            location_requirement_name = data['applicantLocationRequirements']['name']
        except:
            location_requirement_name = ''

            
        data ={
            'description':description,
            'applicant_caption':applicant_caption,
            'posted_time_ago':posted_time_ago,
            'seniority_level':seniority_level,
            'employment_type':employment_type,
            'job_function':job_function,
            'industry':industries,
            'date_posted' : date_posted,
            'description_json' : description,
            'employment_type_json' : employment_type,
            'industry_json' : industry,
            'latitude' : latitude,
            'longitude' : longitude,
            'education_type' : education_type,
            'education_category' : education_category,
            'experience_type' : experience_type,
            'experience_time' : experience_time,
            'job_location_type' : job_location_type,
            'location_requirement_type' : location_requirement_type,
            'location_requirement_name' : location_requirement_name,
            
        }
        job_details.append(data)
        driver.close()

In [49]:
job_details = []
extract_job_details(df['job_link'])

Transform

In [50]:
#add extracted columns to job list. since row count is the same, no common key needed
df3 = pd.concat([df,
           pd.DataFrame(job_details)],
           axis=1)

Load

In [2]:
#save to folder with today's date as filename. these csv's can be saved as backup
today = date.today()
df3.to_csv('{}{}.csv'.format(config.save_path,today))

In [ ]:
#load google cloud credentials
credentials = service_account.Credentials.from_service_account_file(
    '{}'.format(config.service_path)
    ,scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

In [ ]:
#append data to bigquery
job_config = bigquery.LoadJobConfig()
# Make the API request
job_config.create_disposition = "CREATE_IF_NEEDED"
table_id = "linkedin_jobs.data_analyst_chicago"
job = client.load_table_from_dataframe(dataframe=df3,
                                               destination=table_id, 
                                               job_config=job_config)
job.result()

In [3]:
#end timer
end_time = time.perf_counter()
print('Duration: {}'.format(end_time - start_time))